# RLDT


## Step 1: Import the necessary libraries:


In [241]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

## Step 2: Define the environment:


### Step 2.1: Devices


#### _Gloabl variables_


In [242]:
num_IOT_devices = 10

voltages_frequencies_IOT = [
    (10e6  , 1.8),
    (20e6  , 2.3),
    (40e6  , 2.7),
    (80e6  , 4.0),
    (160e6 , 5.0),
]

num_MEC_devices = 5

voltages_frequencies_MEC = [
    (1500e6 ,  1.2),
    (1000e6 ,  1.0),
    (750e6, 0.825),
    (600e6, 0.8),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5
num_dag_generations = 10000


#### _IOT_


In [243]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 8, 16])
    device_info = {
        "id": f"iot {i}",
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [0 for _ in range(cpu_cores)],
        "voltages_frequencies": [
            [
                voltages_frequencies_IOT[i]
                for i in np.random.choice(5, size=3, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "ISL": np.random.randint(10, 21) / 100,
        "capacitance": [np.random.uniform(2, 3) * 1e-10 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([800, 900,1000]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)


#### _MEC_


In [244]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16, 32, 64])
    device_info = {
        "id":f"mec {i}",
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [0 for _ in range(cpu_cores)],
        "voltages_frequencies":[
            [
                voltages_frequencies_MEC[i]
                for i in np.random.choice(4, size=3, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([550, 650, 750]) * 1e-3 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
        "batteryLevel": 100,
        "ISL": 0,
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)


#### _CLOUD_


In [245]:
device_info = [
    {
        "id": 'cloud',
        "number_of_cpu_cores": 1,
        "occupied_cores": [0],
        "voltages_frequencies": [2.8e9, 3.9e9],
        "capacitance": (13.85, 24.28),
        "powerIdle": 0,
        "ISL": 0,
        "batteryLevel": 100,
        "errorRate": 0.1,
        "accetableTasks": [1, 2, 3, 4],
        "handleSafeTask": 0,
    }
]
cloud = pd.DataFrame(device_info)

#### ALL THE DEVICES


In [246]:
devices = pd.concat([IoTdevices,MECDevices,cloud],ignore_index=True)
# devices = pd.concat([IoTdevices,MECDevices],ignore_index=True)
devices

,id,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
0,iot 0,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(80000000.0, 4.0), (40000000.0, 2.7), (16000...",0.14,"[2.6241330046577314e-10, 2.0741287296807164e-1...","[0.001, 0.0007999999999999999, 0.001, 0.001, 0...",4.000000e+10,0.02,"[1, 2, 4, 3]",1
1,iot 1,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(20000000.0, 2.3), (10000000.0, 1.8), (40000...",0.18,"[2.0718434589570324e-10, 2.4411602611126784e-1...","[0.0009, 0.001, 0.0009, 0.001, 0.001, 0.000799...",3.800000e+10,0.02,"[3, 4, 2]",1
2,iot 2,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(80000000.0, 4.0), (10000000.0, 1.8), (20000...",0.19,"[2.6269929610375816e-10, 2.964767898100999e-10...","[0.0009, 0.0007999999999999999, 0.0009, 0.0009...",3.900000e+10,0.05,"[2, 1]",1
3,iot 3,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(160000000.0, 5.0), (80000000.0, 4.0), (1000...",0.12,"[2.4328351728174293e-10, 2.2642026665553688e-1...","[0.0007999999999999999, 0.001, 0.0007999999999...",3.700000e+10,0.03,"[2, 4]",1
4,iot 4,4,"[0, 0, 0, 0]","[[(20000000.0, 2.3), (10000000.0, 1.8), (40000...",0.12,"[2.718953145753326e-10, 2.4392997801620807e-10...","[0.0009, 0.0009, 0.001, 0.0009]",4.000000e+10,0.01,"[3, 4]",1
5,iot 5,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(40000000.0, 2.7), (80000000.0, 4.0), (10000...",0.17,"[2.7535161322322405e-10, 2.655761313188588e-10...","[0.0007999999999999999, 0.0007999999999999999,...",4.000000e+10,0.03,"[4, 2]",1
6,iot 6,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(20000000.0, 2.3), (10000000.0, 1.8), (40000...",0.14,"[2.0939315982073376e-10, 2.9062022872073713e-1...","[0.0009, 0.0009, 0.001, 0.0007999999999999999,...",3.900000e+10,0.01,"[3, 1]",1
7,iot 7,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(80000000.0, 4.0), (40000000.0, 2.7), (20000...",0.11,"[2.5326021542569794e-10, 2.739087450745185e-10...","[0.0007999999999999999, 0.0007999999999999999,...",3.800000e+10,0.05,"[1, 3, 2]",1
8,iot 8,4,"[0, 0, 0, 0]","[[(40000000.0, 2.7), (20000000.0, 2.3), (10000...",0.16,"[2.525992663707574e-10, 2.648301147956866e-10,...","[0.0009, 0.0007999999999999999, 0.000799999999...",3.900000e+10,0.01,"[4, 2, 3]",1
9,iot 9,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(40000000.0, 2.7), (80000000.0, 4.0), (16000...",0.11,"[2.179913210549228e-10, 2.3683417308976047e-10...","[0.001, 0.0007999999999999999, 0.0009, 0.001, ...",4.000000e+10,0.02,"[1, 2]",1


### Step 2.2: Application


#### _helper function : generate_random_dag_


In [247]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):

        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(
                available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}")
                           for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### _Generate task DAGs_


In [248]:
tasks_data = []

start_node_number = 1
for run in range(num_dag_generations):

    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

    random_dag = generate_random_dag(num_nodes)

    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }

    random_dag = nx.relabel_nodes(random_dag, mapping)
    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "job": run,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": int(np.random.uniform(1, 11)*1e6),
            "dataEntrySize":int(np.random.uniform(1, 11)*1e6),
            "returnDataSize":int(np.random.uniform(1, 11)*1e6),
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes

np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)
tasks

,job,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,,
t16427,1363,[],8,2,0,1327805,6832489,1378328,READY
t43827,3652,"[t43824, t43825, t43826]",4,1,0,6533003,3035034,8690090,READY
t12511,1033,[t12507],3,2,0,4784702,5825838,2931392,READY
t28425,2377,"[t28407, t28413, t28414, t28418, t28424]",9,3,0,6832256,8276801,5417647,READY
t44814,3738,[t44813],9,2,0,4397187,6065847,9260350,READY
...,...,...,...,...,...,...,...,...,...
t48070,4013,"[t48066, t48067]",2,4,0,9958944,5172754,1868089,READY
t24112,2002,[t24110],6,4,0,9892477,3576756,4124410,READY
t44158,3677,"[t44150, t44151, t44153]",3,3,1,8651983,9142212,9104367,READY


## Step 4 : DDT


### Step 4.1: Initializing The tree


In [249]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        if depth != max_depth:
            # self.weights = nn.Parameter(torch.zeros(num_input))
            self.weights = nn.Parameter(torch.empty(
                num_input).normal_(mean=0, std=0.1))
            self.bias = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))

        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist
        val = torch.sigmoid(torch.matmul(x, self.weights.t()) + self.bias)
        a = np.random.uniform(0, 1)
        if a < 0.1:
            val = 1 - val
        if val >= 0.5:

            return val * self.right(x)
        else:

            return (1 - val) * self.left(x)

## Step 5: RL


In [250]:
def calc_execution_time(device, task, core, dvfs):
    if device['id'] == "cloud":
        return task["computationalLoad"] / device["voltages_frequencies"][0]
    else:
        return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0]


def calc_power_consumption(device, task, core, dvfs):
    if device['id'] == "cloud":
        return 13.85 * calc_execution_time(device, task, core, dvfs)
    return (
        device["capacitance"][core]
        * (device["voltages_frequencies"][core][dvfs][1] ** 2)
        * device["voltages_frequencies"][core][dvfs][0]
    )
def calc_energy(device, task, core, dvfs):
    return calc_execution_time(device, task, core, dvfs) * calc_power_consumption(device, task, core, dvfs)


def calc_total(device, task, core, dvfs):
    transferRate5g =1e9
    latency5g=5e-3
    transferRateFiber =1e10
    latencyFiber=1e-3

    timeDownMec = task["returnDataSize"] / transferRate5g
    timeDownMec += latency5g
    timeUpMec = task["dataEntrySize"] / transferRate5g
    timeUpMec += latency5g

    alpha = 52e-5
    beta = 3.86412
    powerMec = alpha * 1e9 / 1e6 + beta

    timeDownCC = task["returnDataSize"] / transferRateFiber
    timeDownCC += latencyFiber
    timeUpCC = task["dataEntrySize"] / transferRateFiber
    timeUpCC += latencyFiber

    powerCC = 3.65 
    if device["id"].startswith("mec"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec *  timeTransMec
        totalTime = calc_execution_time(device, task, core, dvfs) + timeTransMec 
        totalEnergy = calc_energy(device, task, core, dvfs) + energyTransMec

    elif device['id'].startswith("cloud"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec * timeTransMec
        
        timeTransCC = timeUpCC+timeDownCC
        energyTransCC =  powerCC * timeTransCC
        
        totalTime = calc_execution_time(device, task, core, dvfs) + timeTransMec +timeTransCC
        totalEnergy = calc_energy(device, task, core, dvfs) + energyTransMec + energyTransCC

    elif device['id'].startswith("iot"):
        totalTime = calc_execution_time(device, task, core, dvfs)
        totalEnergy = calc_energy(device, task, core, dvfs)

    return totalTime , totalEnergy

In [251]:
tasks_copy = tasks.copy()
tasks_copy = tasks_copy.drop(["job","dependency","mobility","status"],axis=1)
taskList = tasks_copy.index.tolist()

In [252]:
class Environment:
    def __init__(self):
        self.feature_size = 5
        self.num_actions = len(devices)
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions,
                         depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

    def execute_action(self, state, action):
        taskList.pop(0)
        device = devices.iloc[action]

        checkIfSutible = False
        if not state['safe'] or device["handleSafeTask"]:
            checkIfSutible = True

        if (checkIfSutible):
            for coreIndex in range(len(device["occupied_cores"])):
                if device["occupied_cores"][coreIndex] == 0:
                    total_t, total_e = calc_total(device, state, coreIndex,np.random.randint(0,3))
                    reward = -1 * total_t + -1 * total_e
                    print(f"device {device['id']} ////// time {total_t}  ////// energy {total_e} ")
                    return (tasks_copy.loc[taskList[0]], reward)

        return (tasks_copy.loc[taskList[0]], 1)

    def train(self, num_epoch, num_episodes):
        for i in range(num_epoch):
            total_loss = 0
            for j in range(num_episodes):
                state = tasks_copy.loc[taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)
                
                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)
                action_index = torch.multinomial(action_probabilities, 1).item()

                next_state,reward = self.execute_action(state, action_index)
                loss = (output[action_index] * reward)
                total_loss += loss
                

            self.optimizer.zero_grad()
            avg_loss = total_loss/num_episodes
            avg_loss.backward()
            self.optimizer.step()
            print(f"Avg loss {avg_loss}")



env = Environment()
tree = env.agent
env.train(1, 100)

print('///////////////////')

for name, param in env.agent.named_parameters():
    if "prob_dist" or "bias" not in name:
        print(name,param)
        pass

device iot 6 ////// time 0.06639025  ////// energy 0.0014707960754058197 
device iot 6 ////// time 0.6533003  ////// energy 0.004432210297774199 
device iot 4 ////// time 0.2392351  ////// energy 0.006881962313273489 
device iot 1 ////// time 0.1708064  ////// energy 0.010319261014666037 
device mec 2 ////// time 0.032654842  ////// energy 0.11575732532490371 
device iot 6 ////// time 0.477407  ////// energy 0.010576377434506817 
device iot 2 ////// time 1.0303199  ////// energy 0.008769523724730903 
device mec 4 ////// time 0.029229471  ////// energy 0.10242550949279444 
device mec 3 ////// time 0.035547193  ////// energy 0.1373316283063076 
device iot 3 ////// time 0.068418025  ////// energy 0.06657991106988087 
device iot 1 ////// time 0.9354137  ////// energy 0.006279219648674698 
device mec 4 ////// time 0.024973591333333333  ////// energy 0.10868488206459456 
device mec 3 ////// time 0.026762193  ////// energy 0.11168596882011872 
device iot 9 ////// time 0.05198599375  ////// en